# 필요한 라이브러리 import

In [91]:
import os
import json
import numpy as np
import pandas as pd
from gensim import corpora
from gensim.models import Word2Vec
from sklearn.decomposition import TruncatedSVD
from scipy import sparse as spr
import configparser
import pymysql
from itertools import chain
from collections import defaultdict
import hnswlib
import warnings
import tqdm
import pickle
import matplotlib.pyplot as plt
warnings.filterwarnings(action='ignore')

# 데이터 로드

In [92]:
with open('data/train.json',encoding='utf-8-sig') as f:
    train_dict = json.load(f)
    
train_df = pd.DataFrame.from_dict(train_dict)

# 태그와 id를 맞추기 위한 데이터베이스 연결

In [93]:
# config parser 객체 생성
config = configparser.ConfigParser()

# parser객체로 config.ini 파일 읽기
config.read('config.ini')

# 변수저장

user = config['DEFAULT']['ADMIN_USER_NAME']
passwd = config['DEFAULT']['ADMIN_PASSWORD']
host = config['DEFAULT']['RDS_ENDPOINT']
port = config['DEFAULT']['PORT']
database = config['DEFAULT']['DEFAULT_DATABASE']

In [94]:
melon_recom_db = pymysql.connect(
    host = host,
    user = user,
    password = passwd,
    database = database,
    port = int(port),
    cursorclass = pymysql.cursors.DictCursor
)

In [95]:
melon_cursor = melon_recom_db.cursor()
melon_cursor.execute('select * from tag')
tag_db_list=melon_cursor.fetchall()

# tag 딕셔너리 만들기 

In [96]:
tag_to_id = {}
id_to_tag = {}

for tag_db in tag_db_list:
    tag_to_id[tag_db['tag']] = tag_db['tag_id']
    
for tag_db in tag_db_list:
    id_to_tag[tag_db['tag_id']] = tag_db['tag']
    
train_df['new_tags_id'] = train_df['tags'].map(lambda x : [tag_to_id[v] for v in x])

# tag 전처리 빈도수로 컷

In [97]:
# train dataframe tag 컬럼의 모든 tag들 (중복포함)
tags_all = train_df['tags'].tolist()

# 태그의 빈도수를 가진 dict, Counter 써도 됨
tags_frequency = defaultdict(int)

# 특정 tag가 나올 때마다 1더하기
for tags in tags_all:
    for tag in tags:
        tags_frequency[tag] += 1

In [98]:
tag_freq = pd.DataFrame().from_dict(tags_frequency,orient="index")

In [99]:
tag_freq.columns=['빈도']
tag_freq.head()

,빈도
락,4007
추억,6520
회상,4579
까페,2856
잔잔한,10218


In [100]:
tag_freq.describe()

,빈도
count,29160.000000
mean,16.335082
std,247.011075
min,1.000000
25%,1.000000
50%,1.000000
75%,3.000000
max,16465.000000


In [137]:
# tag 중에 빈도수가 1번 이상 나오고, 플레이리스트 당 tag가 1개 이상인 것들만 포함
tags_more_than_one = [[tag for tag in tags if tags_frequency[tag] > 16 ] for tags in tags_all if len(tags) > 1]

In [145]:
def filter_func(x):
    temp = []
    for tag in x:
        if tags_frequency[tag] > 16:
            temp.append(tag)
        else:
            pass
    return temp
            

train_df['tag_mto'] = train_df['tags'].map(filter_func)

In [151]:
train_df['new_tags_mto_id'] = train_df['tag_mto'].map(lambda x : [tag_to_id[v] for v in x])
train_df.head(5)

,tags,id,plylst_title,songs,like_cnt,updt_date,new_tags_id,tag_mto,new_tags_mto_id
0,[락],61281,여행같은 음악,"[525514, 129701, 383374, 562083, 297861, 13954...",71,2013-12-19 18:36:19.000,[25304],[락],[25304]
1,"[추억, 회상]",10532,요즘 너 말야,"[432406, 675945, 497066, 120377, 389529, 24427...",1,2014-12-02 16:19:42.000,"[17676, 152]","[추억, 회상]","[17676, 152]"
2,"[까페, 잔잔한]",76951,"편하게, 잔잔하게 들을 수 있는 곡.-","[83116, 276692, 166267, 186301, 354465, 256598...",17,2017-08-28 07:09:34.000,"[21656, 26271]","[까페, 잔잔한]","[21656, 26271]"
3,"[연말, 눈오는날, 캐럴, 분위기, 따듯한, 크리스마스캐럴, 겨울노래, 크리스마스,...",147456,크리스마스 분위기에 흠뻑 취하고 싶을때,"[394031, 195524, 540149, 287984, 440773, 10033...",33,2019-12-05 15:15:18.000,"[19772, 22488, 10907, 19970, 8380, 16144, 2196...","[연말, 눈오는날, 캐럴, 분위기, 따듯한, 겨울노래, 크리스마스, 겨울왕국]","[19772, 22488, 10907, 19970, 8380, 21963, 170,..."
4,[댄스],27616,추억의 노래 ㅋ,"[159327, 553610, 5130, 645103, 294435, 100657,...",9,2011-10-25 13:54:56.000,[13941],[댄스],[13941]


# Vectorization with w2v

In [147]:
# 벡터 크기 100으로 해서 window=4 주고 Word2Vec 학습
w2v_model = Word2Vec(sentences=train_df['tag_mto'].tolist(),vector_size=100,window=100,min_count=1,workers=4,sg=1)

In [148]:
Word2Vec.save(w2v_model,'./models/w2v_model')

In [149]:
# w2v 모델 불러오기
w2v_model = Word2Vec.load('./models/w2v_model')

In [150]:
# 락과 관련된 태그 10개
sims = w2v_model.wv.most_similar(['백예린'], topn=10)
sims

[('Crush', 0.8776894807815552),
 ('딘', 0.8688459992408752),
 ('지코', 0.8574314117431641),
 ('DEAN', 0.8524003028869629),
 ('자이언티', 0.8500043153762817),
 ('크러쉬', 0.8491696715354919),
 ('헤이즈', 0.8478885889053345),
 ('넉살', 0.8439952731132507),
 ('폴킴', 0.8380653858184814),
 ('주관적', 0.835231363773346)]

# Vectorization with SVD

## 희소행렬생성 태그,음악 행렬

In [13]:
A = spr.load_npz('./data/tag_song_csr.npz')

In [14]:
A = A.astype(float)

In [15]:
A.shape

(29160, 707989)

In [153]:
from itertools import repeat

row = []
col = []
dat = []

for tags,song_id in zip(train_df['new_tags_mto_id'],train_df['songs']):
    for tag in tags:
        row.extend(list(repeat(tag,len(song_id))))
        col.extend(song_id)
        dat.extend(list(repeat(1,len(song_id))))

In [154]:
A = spr.csr_matrix((dat, (row, col)), shape=(29160, 707989))

In [156]:
A = A.astype(float)

## SVD 만들기

In [157]:
u,s,vt = spr.linalg.svds(A,k=100)

In [158]:
svd_vectors = u

In [159]:
svd_vectors.shape

(29160, 100)

In [160]:
data_len,dim = svd_vectors.shape

In [161]:
p = hnswlib.Index(space='cosine', dim=dim)  

In [162]:
p.init_index(max_elements=data_len, ef_construction=100, M=100)

In [163]:
p.add_items(svd_vectors,np.arange(data_len))

In [164]:
#pickle.dump(p,'./data/svd_knn_model.pickle')
with open('./data/svd_knn_model.pickle', 'wb') as f:
    pickle.dump(p,f)

with open('./data/svd_knn_model.pickle', 'rb') as f:
    p = pickle.load(f)

In [21]:
tag_to_id['백예린'] , tag_to_id['아이유']

(3113, 10095)

In [165]:
svd_vectors[3113]

array([-1.18716269e-02,  8.52625424e-03, -2.90227060e-03, -1.60118010e-02,
        3.19737582e-03,  1.74909994e-02, -4.37572772e-03, -6.04426009e-03,
       -6.76970216e-03, -2.86884851e-03,  3.66130285e-03, -2.59979279e-03,
        9.70578285e-03, -2.39198710e-03,  1.05910212e-02,  3.51443472e-03,
       -1.03440406e-04, -1.26130641e-02,  2.82887292e-03,  1.06785371e-02,
        6.55941492e-03, -1.80890384e-03, -4.04610329e-03,  2.60053013e-03,
        1.19107603e-03,  1.10479948e-03,  3.01075158e-03,  2.98518575e-03,
       -1.47336950e-04, -2.22012590e-03,  1.91433071e-03, -8.74203348e-03,
       -3.40751100e-03, -2.58391611e-03,  1.07083377e-02, -2.50993529e-03,
        6.37875185e-03, -4.22030118e-03,  4.27163367e-03,  1.55716935e-03,
       -6.59646890e-04,  1.76368120e-04, -5.92747226e-03, -3.13774583e-03,
        9.29140454e-04, -4.16523053e-03,  1.76529584e-03,  5.00906698e-04,
        4.55927277e-03, -6.24603202e-04, -4.29213230e-03,  2.86331909e-03,
        2.21578242e-03, -

In [167]:
labels, distances = p.knn_query(svd_vectors[3113], k = 11)
print(labels, distances)

[[ 3113 25025 12585 25071 18914  1895 25923 13182  5641 22457 14498]] [[-2.3841858e-07  3.4221452e-01  3.7780726e-01  4.0097880e-01
   4.0919149e-01  4.2138457e-01  4.3166775e-01  4.3826419e-01
   4.5196521e-01  4.5440799e-01  4.6365178e-01]]


In [168]:
#id_to_tag[24585]

[id_to_tag[x] for x in labels[0][1:]]

['음색깡패', '고막여친', '조용한음악', '여자솔로', '여성보컬', '자이언티', '국내', '여자가수', '여자보컬', '그루비한']

In [169]:
train_df['tag_cnt'] = train_df['tags'].map(len)

In [175]:
origin_tags = train_df[train_df['tag_cnt']>9]['tag_mto'].tolist()

train_tags = []
test_tags = []

for tags in origin_tags:

    train_tags.append(tags[:5])
    test_tags.append(tags[5:])

In [176]:
cal_tag_hit_df = pd.DataFrame(columns=['태그원본','예측용태그','검증용태그','w2v_예측결과','svd_예측결과','w2v_히트','svd_히트'])
cal_tag_hit_df['태그원본'] = origin_tags
cal_tag_hit_df.head()

,태그원본,예측용태그,검증용태그,w2v_예측결과,svd_예측결과,w2v_히트,svd_히트
0,"[연말, 눈오는날, 캐럴, 분위기, 따듯한, 겨울노래, 크리스마스, 겨울왕국]",NaN,NaN,NaN,NaN,NaN,NaN
1,"[운동, 드라이브, Pop, 트로피컬하우스, 힐링, 기분전환, 2017, 팝, 트렌...",NaN,NaN,NaN,NaN,NaN,NaN
2,"[새해, 여행, 기분전환]",NaN,NaN,NaN,NaN,NaN,NaN
3,"[여름, 일렉트로니카, 매장음악, 취향저격, 댄스, 드라이브, 여행, 기분전환, 일...",NaN,NaN,NaN,NaN,NaN,NaN
4,"[헬스, 스포츠, 피트니스, 운동, 다이어트, 런닝, 필라테스, 산책, 요가]",NaN,NaN,NaN,NaN,NaN,NaN


In [177]:
cal_tag_hit_df['예측용태그'] = train_tags
cal_tag_hit_df['검증용태그'] = test_tags
cal_tag_hit_df

,태그원본,예측용태그,검증용태그,w2v_예측결과,svd_예측결과,w2v_히트,svd_히트
0,"[연말, 눈오는날, 캐럴, 분위기, 따듯한, 겨울노래, 크리스마스, 겨울왕국]","[연말, 눈오는날, 캐럴, 분위기, 따듯한]","[겨울노래, 크리스마스, 겨울왕국]",NaN,NaN,NaN,NaN
1,"[운동, 드라이브, Pop, 트로피컬하우스, 힐링, 기분전환, 2017, 팝, 트렌...","[운동, 드라이브, Pop, 트로피컬하우스, 힐링]","[기분전환, 2017, 팝, 트렌드, 일렉]",NaN,NaN,NaN,NaN
2,"[새해, 여행, 기분전환]","[새해, 여행, 기분전환]",[],NaN,NaN,NaN,NaN
3,"[여름, 일렉트로니카, 매장음악, 취향저격, 댄스, 드라이브, 여행, 기분전환, 일...","[여름, 일렉트로니카, 매장음악, 취향저격, 댄스]","[드라이브, 여행, 기분전환, 일렉, 신나는]",NaN,NaN,NaN,NaN
4,"[헬스, 스포츠, 피트니스, 운동, 다이어트, 런닝, 필라테스, 산책, 요가]","[헬스, 스포츠, 피트니스, 운동, 다이어트]","[런닝, 필라테스, 산책, 요가]",NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...
12570,"[출근길, 노동요, 연주곡, 카페, 매장, 모닝콜, 휴식, 피아노, 쌀쌀한, 아침]","[출근길, 노동요, 연주곡, 카페, 매장]","[모닝콜, 휴식, 피아노, 쌀쌀한, 아침]",NaN,NaN,NaN,NaN
12571,"[감성, 잔잔한, 취향저격, 팝송음악, Pop, 휴식, 팝송, 기분전환, 힐링, 신나는]","[감성, 잔잔한, 취향저격, 팝송음악, Pop]","[휴식, 팝송, 기분전환, 힐링, 신나는]",NaN,NaN,NaN,NaN
12572,"[감성, 발라드, 드라이브, 댄스, 추억, 이별노래, 기분전환, 추천노래, 옛날노래]","[감성, 발라드, 드라이브, 댄스, 추억]","[이별노래, 기분전환, 추천노래, 옛날노래]",NaN,NaN,NaN,NaN
12573,"[클럽, 홍대, 운동, 열정, EDM, 20대, 라운지, 느낌, 일렉]","[클럽, 홍대, 운동, 열정, EDM]","[20대, 라운지, 느낌, 일렉]",NaN,NaN,NaN,NaN


In [178]:
def tag_wv2(x):
    sims = w2v_model.wv.most_similar(x, topn=10)
    return [x[0] for x in sims]

In [179]:
cal_tag_hit_df['w2v_예측결과'] = cal_tag_hit_df['예측용태그'].map(tag_wv2)
cal_tag_hit_df.head()

ValueError: cannot compute similarity with no input

In [63]:
def tag_svd(x):
    target_ids = [tag_to_id[t] for t in x]
    
    vectors = np.zeros((100,1))
    
    for id in target_ids:
        vectors = vectors+svd_vectors[id]
    
    labels, distances = p.knn_query(vectors/len(target_ids), k = 11)
    
    ids = [l for l in labels[0][1:]]
    
    return [id_to_tag[tag] for tag in ids]

In [64]:
cal_tag_hit_df['svd_예측결과'] = cal_tag_hit_df['예측용태그'].map(tag_svd)
cal_tag_hit_df.head()

,태그원본,예측용태그,검증용태그,w2v_예측결과,svd_예측결과,w2v_히트,svd_히트
0,"[연말, 눈오는날, 캐럴, 분위기, 따듯한, 크리스마스캐럴, 겨울노래, 크리스마스,...","[연말, 눈오는날, 캐럴, 분위기, 따듯한]","[크리스마스캐럴, 겨울노래, 크리스마스, 겨울왕국, 크리스마스송]","[추운날, 춥다, 12월, 훈훈한, 찬바람, 한겨울, 추위, 겨울감성, 첫눈, 추운날씨]","[혼술남녀, 하석진, 여름밤바다, 멜팅, 레전드째즈보컬, 브라스팝, 블라디보스톡, ...",NaN,NaN
1,"[운동, 드라이브, Pop, 트로피컬하우스, 힐링, 기분전환, 2017, 팝, 트렌...","[운동, 드라이브, Pop, 트로피컬하우스, 힐링]","[기분전환, 2017, 팝, 트렌드, 일렉]","[YOLO, 팝송을, 팝송음악, 미디엄템포, popmusic, 신나고, 팝뮤직, 카...","[김래원, 이광수, 편안히잠잘수있는클래식, 머리가맑아지는클래식, 우리아이피아노클래식...",NaN,NaN
2,"[새해, 여행, 프로필음악, 카카오톡, 기분전환, 소원, 프로필, 소망, 다짐, 카톡]","[새해, 여행, 프로필음악, 카카오톡, 기분전환]","[소원, 프로필, 소망, 다짐, 카톡]","[기분좋게, 프로필, 가볍게들을수있는곡, 최신음악, 발걸음, 흥얼흥얼, 연초, 즐거...","[비행기, 프로여행러, 아프리카청춘이다, 동남아, 아이슬란드, 출국, 즉흥, 유럽,...",NaN,NaN
3,"[여름, 일렉트로니카, 매장음악, 취향저격, 댄스, 드라이브, 여행, 기분전환, 일...","[여름, 일렉트로니카, 매장음악, 취향저격, 댄스]","[드라이브, 여행, 기분전환, 일렉, 신나는]","[취항저격, YOLO, 해외일렉트로니카, 미디엄템포, 놀자, 트로피컬, 신나고, 매...","[딥하우스, 하우스, 일렉트로팝, 내적런웨이, 감각적비트, deephouse, 패피...",NaN,NaN
4,"[헬스, 스포츠, 피트니스, 운동, 다이어트, 런닝, 레깅스, 필라테스, 산책, 요가]","[헬스, 스포츠, 피트니스, 운동, 다이어트]","[런닝, 레깅스, 필라테스, 산책, 요가]","[런닝, 조깅, 달리기, 유산소, 웨이트, 몸매, 홈트, 헬스장, 휘트니스, 살빼기...","[헬스, 다이어트, 런닝, 런닝머신, 헬스장, 운동, 운동할때듣는음악, 버닝, 달리...",NaN,NaN


In [65]:
val_tag = cal_tag_hit_df['검증용태그'].tolist()
w2v_tag = cal_tag_hit_df['w2v_예측결과'].tolist()
svd_tag = cal_tag_hit_df['svd_예측결과'].tolist()

In [66]:
w2v_hit = []

for val,w2v in zip(val_tag,w2v_tag):
    if len(val) == len(set(val)-set(w2v)):
        w2v_hit.append(0)
    else:
        w2v_hit.append(1)
        
w2v_hit[:5]

[0, 0, 1, 0, 1]

In [67]:
svd_hit = []

for val,svd in zip(val_tag,svd_tag):
    if len(val) == len(set(val)-set(svd)):
        svd_hit.append(0)
    else:
        svd_hit.append(1)
             
svd_hit[:5]

[0, 0, 0, 0, 1]

In [68]:
sum(w2v_hit)

cal_tag_hit_df['w2v_히트'] = w2v_hit

In [69]:
sum(svd_hit)

cal_tag_hit_df['svd_히트'] = svd_hit

In [70]:
cal_tag_hit_df.head()

,태그원본,예측용태그,검증용태그,w2v_예측결과,svd_예측결과,w2v_히트,svd_히트
0,"[연말, 눈오는날, 캐럴, 분위기, 따듯한, 크리스마스캐럴, 겨울노래, 크리스마스,...","[연말, 눈오는날, 캐럴, 분위기, 따듯한]","[크리스마스캐럴, 겨울노래, 크리스마스, 겨울왕국, 크리스마스송]","[추운날, 춥다, 12월, 훈훈한, 찬바람, 한겨울, 추위, 겨울감성, 첫눈, 추운날씨]","[혼술남녀, 하석진, 여름밤바다, 멜팅, 레전드째즈보컬, 브라스팝, 블라디보스톡, ...",0,0
1,"[운동, 드라이브, Pop, 트로피컬하우스, 힐링, 기분전환, 2017, 팝, 트렌...","[운동, 드라이브, Pop, 트로피컬하우스, 힐링]","[기분전환, 2017, 팝, 트렌드, 일렉]","[YOLO, 팝송을, 팝송음악, 미디엄템포, popmusic, 신나고, 팝뮤직, 카...","[김래원, 이광수, 편안히잠잘수있는클래식, 머리가맑아지는클래식, 우리아이피아노클래식...",0,0
2,"[새해, 여행, 프로필음악, 카카오톡, 기분전환, 소원, 프로필, 소망, 다짐, 카톡]","[새해, 여행, 프로필음악, 카카오톡, 기분전환]","[소원, 프로필, 소망, 다짐, 카톡]","[기분좋게, 프로필, 가볍게들을수있는곡, 최신음악, 발걸음, 흥얼흥얼, 연초, 즐거...","[비행기, 프로여행러, 아프리카청춘이다, 동남아, 아이슬란드, 출국, 즉흥, 유럽,...",1,0
3,"[여름, 일렉트로니카, 매장음악, 취향저격, 댄스, 드라이브, 여행, 기분전환, 일...","[여름, 일렉트로니카, 매장음악, 취향저격, 댄스]","[드라이브, 여행, 기분전환, 일렉, 신나는]","[취항저격, YOLO, 해외일렉트로니카, 미디엄템포, 놀자, 트로피컬, 신나고, 매...","[딥하우스, 하우스, 일렉트로팝, 내적런웨이, 감각적비트, deephouse, 패피...",0,0
4,"[헬스, 스포츠, 피트니스, 운동, 다이어트, 런닝, 레깅스, 필라테스, 산책, 요가]","[헬스, 스포츠, 피트니스, 운동, 다이어트]","[런닝, 레깅스, 필라테스, 산책, 요가]","[런닝, 조깅, 달리기, 유산소, 웨이트, 몸매, 홈트, 헬스장, 휘트니스, 살빼기...","[헬스, 다이어트, 런닝, 런닝머신, 헬스장, 운동, 운동할때듣는음악, 버닝, 달리...",1,1


In [71]:
sum(w2v_hit)/len(w2v_hit)

0.24795228628230617

In [72]:
sum(svd_hit)/len(svd_hit)

0.10298210735586481